# baseline code for hackathon

you can change some parts of the baseline code to improve the performance. 

put images in the class directories for train, valid and test.

for example, with 'HDH' and 'OH' classes (we have 5 classes)
* ./drive/My Drive/public/data/HGU_buildings/train/HDH/*.jpg
* ./drive/My Drive/public/data/HGU_buildings/train/OH/*.jpg
* ./drive/My Drive/public/data/HGU_buildings/valid/HDH/*.jpg
* ./drive/My Drive/public/data/HGU_buildings/valid/OH/*.jpg
* ./drive/My Drive/public/data/HGU_buildings/test/HDH/*.jpg
* ./drive/My Drive/public/data/HGU_buildings/test/OH/*.jpg

submit your model and transform files including your team name in the file names.  
* e.g., 'teamX.model' and 'teamX.transform' in the baseline code (when the team name is 'teamX') 
* share the two files in your google drive to TAs 
* for the same accuracy, the team with a smaller model wins.  

have fun! 
(by hchoi@handong.edu, Nov. 29, 2019) 

In [249]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os
import warnings
warnings.filterwarnings("ignore")

# try to change the network architecture
* different kernel size and numbers

In [0]:
class MyCNN(nn.Module):
    def __init__(self, output_dim=10):
        super(MyCNN,self).__init__()

        self.output_dim=output_dim

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3,256,3,padding=1), # try with different kernels
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256,256,3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(10,10), # 32 x (25x25)
            
            nn.Conv2d(256,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2,2) # 16 x (5x5) 
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(16*5*5,100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,output_dim)
        )       
        
    def forward(self,x):
        out = self.cnn_layers(x)
        out = out.view(out.shape[0],-1)
        out = self.fc_layer(out)

        return out


# try to change the learning rate

In [328]:
learning_rate = 0.00005
output_dim=5

model = MyCNN(output_dim=output_dim).cuda()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_list = list(model.children())
print(param_list)

[Sequential(
  (0): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=400, out_features=100, bias=True)
  (1): BatchNorm1d(100, eps=1e-05, moment

In [0]:
import torchvision.transforms.functional as TF
import random

def my_segmentation_transforms(image):
    if random.random() > 0.5:
        angle = random.randint(-30, 30)
        image = TF.rotate(image, angle)
        #segmentation = TF.rotate(segmentation, angle)
    # more transforms ...
    return image

class MyRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return TF.rotate(x, angle)

rotation_transform = MyRotationTransform(angles=[-30, -15, 0, 15, 30])

# try to change transform function
* https://pytorch.org/docs/stable/torchvision/transforms.html

In [0]:
import random
input_size=(100, 100)

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3]),
        
        #transforms.RandomErasing()
        
        # transforms.RandomErasing(scale=(x_scale,y_scale))
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.3, 0.3, 0.3]),
        #transforms.RandomErasing()
    ]),
}
test_transform = data_transforms['valid']

**data loader**

In [331]:
batch_size = 64 # try with different batch_size
#data_dir = './drive/My Drive/public/data/HGU_buildings/'
data_dir = './drive/My Drive/sample_code_and_train_valid/'
train_dir = 'train'
valid_dir = 'valid2'

train_set = datasets.ImageFolder(data_dir+train_dir, data_transforms['train'])
valid_set = datasets.ImageFolder(data_dir+valid_dir, data_transforms['valid'])


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)          
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)

train_size = len(train_set)
valid_size = len(valid_set)

class_names = train_set.classes

print(class_names) 
print(f'Train image size: {train_size}')
print(f'Validation image size: {valid_size}')

['ANH', 'HDH', 'Hyoam', 'NTH', 'OH']
Train image size: 4186
Validation image size: 200


**training**

In [332]:
result_dir = 'drive/My Drive/public/real_pro_result/'
num_epoch = 3 # try with different epochs and find the best epoch

if not os.path.exists(result_dir):
    os.mkdir(result_dir)    
    
for i in range(num_epoch):
    model.train()
    for j, [image,label] in enumerate(train_loader):
        x = image.cuda()
        y_= label.cuda()
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 30 == 0:
            print(i,j, loss.data.cpu())
    
    model.eval()
    hits = 0
    for k,[image,label] in enumerate(valid_loader):
        x = image.cuda()
        y_= label.cuda()

        output = model(x)
        y_est = output.argmax(1)
        hits = hits + sum(y_est == y_).cpu()
    print('Hits', hits, 'Accuracy', hits/(valid_size+0.0))    

torch.save(model, result_dir + 'Team02.model')
#torch.save(test_transform, result_dir + 'teamX.transform')
print('training is done by max_epochs', num_epoch)

0 0 tensor(1.6878)
0 30 tensor(0.8844)
0 60 tensor(0.7007)
Hits tensor(156) Accuracy tensor(0.7800)
1 0 tensor(0.6667)
1 30 tensor(0.4613)
1 60 tensor(0.4307)
Hits tensor(182) Accuracy tensor(0.9100)
2 0 tensor(0.4076)
2 30 tensor(0.3530)
2 60 tensor(0.3220)
Hits tensor(186) Accuracy tensor(0.9300)
training is done by max_epochs 3



---
training is done! 
* submit your code, model file and transform file.
* you can put the files in a directory, and share the directory to TAs. 
---

**TA will evaluate your work based on teamX.model and teamX.transform as below**

In [334]:
test_batch_size = 10
result_dir = 'drive/My Drive/public/real_pro_result/'
model_name = 'Team02.model'

model = torch.load(result_dir + model_name)
model.cuda()
model.eval()

#transform_name = 'teamX.transform'
#test_transform = torch.load(result_dir + transform_name)

test_dir = './drive/My Drive/sample_code_and_train_valid/valid2'
test_set = datasets.ImageFolder(test_dir, test_transform)
              
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size,
                                          shuffle=False, num_workers=4)

hits = 0
for k,[image,label] in enumerate(test_loader):
    x = image.cuda()
    y_= label.cuda()
  
    output = model(x)
    y_est = output.argmax(1)
    print('Target', label.numpy(), 'Prediction', y_est.cpu().numpy())
    hits = hits + sum(y_est == y_)
print('hits, accuracy', hits, hits/(len(test_set)+0.0))

Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 1 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 3 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 4 1 1 1 1 1]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 0 4 0 2 3 2 0]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 2 3 3 3 3]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 2 3 3 3 4 4]
Target [

The end! (hchoi@handong.edu)

---

